In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("HW_3_robot_arm.ipynb")

# Robot arm

Part 1: Forward kinematics

Part 2: Optimization

Slides: https://docs.google.com/presentation/d/17aiTBmPZidR6op7TvqYRzYatuc_NETYA1BhgpSHQ-FM/edit?usp=sharing

In [ ]:
# The usual imports
import numpy as np
import matplotlib.pyplot as plt

# These are the routines used in the lab and/or you need to fill in for the homework (lab 5, week 5)
from arm_routines import get_matrices_all_links, create_arm_geometry, set_angles_of_arm_geometry, \
                         plot_complete_arm, get_gripper_location


In [ ]:
# This bit of code tells Jupyter notebooks to also look in the week_6_optimization folder for files
import os
import sys
sys.path.insert(0, os.path.abspath('../Week_6_optimization'))

# These are the routines used in the lab and/or you need to fill in for the homework (lab 6, week 6)
from arm_optimization_routines import distance_to_goal, do_fmin

In [ ]:
# These commands will force JN to actually re-load the external file when you re-execute the import command
%load_ext autoreload
%autoreload 2

# Doing forward kinematics

TODO: Edit the step 3 functions in arm_routines.py (get_matrix_base, get_matrix_link, get_matrices_all_links)

The three functions build the matrices you need to move each component to the end of the previous one. There's a "special" one for figuring out how to place the start of the arm on the base. For every other link the calculation for the matrix is the same. The third function does the for loops that call the second function for each link.

In [ ]:
    # Create the arm geometry (done in Lab 5)
    base_size_param = (1.0, 0.5)
    link_sizes_param = [(0.5, 0.25), (0.3, 0.1), (0.2, 0.05)]
    palm_width_param = 0.1
    finger_size_param = (0.075, 0.025)


    # This function calls each of the set_transform_xxx functions, and puts the results
    # in a list (the gripper - the last element - is a list)
    arm_geometry = create_arm_geometry(base_size_param, link_sizes_param, palm_width_param, finger_size_param)

In [ ]:
## Check the combined link/gripper/finger rotations
# Several different angles to check your routines with 
# Pass the one you want to check into set_angles_of_arm_geometry below
angles_none = [0.0, 0.0, 0.0, [0.0, 0.0, 0.0]]
angles_check_link_0 = [np.pi/4, 0.0, 0.0, [0.0, 0.0, 0.0]]
angles_check_link_0_1 = [np.pi/4, -np.pi/4, 0.0, [0.0, 0.0, 0.0]]
angles_check_wrist = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [np.pi/3.0, 0.0, 0.0]]
angles_check_fingers = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [0.0, np.pi/4.0, -np.pi/4.0]]

In [ ]:
# Use this cell to visually check the results
# With angles_none it should point straight up
# If you do NOT have the gripper function done, this will not show the green + (see next question)
set_angles_of_arm_geometry(arm_geometry, angles_none)
matrices = get_matrices_all_links(arm_geometry)

fig, axs = plt.subplots(1, 1, figsize=(6, 6))
plot_complete_arm(axs, arm_geometry, matrices)

In [ ]:
# Don't change this one
angles_check = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [np.pi/3.0, np.pi/4.0, -np.pi/4.0]]
set_angles_of_arm_geometry(arm_geometry, angles_check)
matrices = get_matrices_all_links(arm_geometry)
print(matrices)

In [ ]:
mat_check_base = np.identity(3)
assert(np.all(np.isclose(matrices[0], mat_check_base, atol=0.01)))

In [ ]:
mat_check_link_1 = np.array([[ 0.0, -1.0,  0.0], \
                             [ 1.0,  0.0,  0.5], \
                             [ 0.0,  0.0,  1.0]])
assert(np.all(np.isclose(matrices[1], mat_check_link_1, atol=0.01)))

In [ ]:
mat_check_link_2 = np.array([[ -1.0, 0.0,  -0.5], \
                             [ 0.0,  -1.0,  0.5], \
                             [ 0.0,  0.0,  1.0]])
assert(np.all(np.isclose(matrices[2], mat_check_link_2, atol=0.01)))

In [ ]:
mat_check_wrist = np.array([[ 1.0, 0.0,  -0.5121], \
                            [ 0.0, 1.0,   0.7121], \
                            [ 0.0,  0.0,  1.0]])
assert(np.all(np.isclose(matrices[-1], mat_check_wrist, atol=0.01)))

In [ ]:
grader.check("step3")

# Gripper location

TODO: edit get_gripper_location to return the x,y location of the gripper.

A reminder that the grasp location is offset from the palm by a bit (["grasp"] key in palm object)

In [ ]:
# Check the gripper function
# As in the previous problem, you can use the "simpler" angles to check your function
angles_gripper_check = [np.pi/3, -np.pi/6, 3.0 * np.pi/6, [-np.pi/4, np.pi/4.0, -np.pi/4.0]]

# Actually set the matrices
set_angles_of_arm_geometry(arm_geometry, angles_gripper_check)
matrices = get_matrices_all_links(arm_geometry)

# Now actually plot - the grasper grip location should show up as a green cross
fig, axs = plt.subplots(1, 1, figsize=(8, 8))
plot_complete_arm(axs, arm_geometry, matrices)

In [ ]:
# Now actually plot - the grasper grip location should show up as a green cross
fig, axs = plt.subplots(1, 1, figsize=(8, 8))
plot_complete_arm(axs, arm_geometry, matrices)

In [ ]:
# Numerical check - don't change these
set_angles_of_arm_geometry(arm_geometry, angles_gripper_check)

grasp_loc = get_gripper_location(arm_geometry)
print(f"Grasp loc is {grasp_loc}, expected (-0.8106, 0.92437)")
assert(np.isclose(grasp_loc[0], -0.8106, atol=0.01) and np.isclose(grasp_loc[1], 0.92437, atol=0.01))

In [ ]:
grader.check("gripper_loc")

# FMin optimization

Note: func_for_fmin from the Lab should be working first

TODO: Edit do_fmin to call func_for_fmin with the appropriate angles, etc


In [ ]:
# Set the starting angles of the arm
#. Note: It really shouldn't matter what these are for the final result, as long as they're "reasonable"
angles_start = [np.pi/6.0, -np.pi/4, 1.5 * np.pi/4, [np.pi/3.0, -np.pi/8.0, np.pi/6.0]]
set_angles_of_arm_geometry(arm_geometry, angles_start)

# Do the optimization
target = np.array([0.55, 1.15])
angles_optimized = do_fmin(angles_start, arm_geometry, target)

# Make sure the angles of the arm are set to the optimized one
set_angles_of_arm_geometry(arm_geometry, angles_optimized)

# Check that this is close to zero
d_to_target = distance_to_goal(arm_geometry, target)
print(f"Distance to goal {d_to_target}, should be close to zero if target reachable")

assert(np.isclose(d_to_target, 0.0, atol=0.01))

In [ ]:
# Plot arm with target
fig, axs = plt.subplots(1, 1, figsize=(8, 8))
plot_complete_arm(axs, arm_geometry, get_matrices_all_links(arm_geometry))
axs.plot(target[0], target[1], '+g', markersize=20)

In [ ]:
grader.check("optimization")

# Generalization

If nothing has been "hardwired" in, this should just work - changing the geometry, the starting angles, the target point. However, if you've hardwired something in, it probably won't...

In [ ]:
# Create the arm geometry
base_size_param = (0.5, 0.25) # squished
link_sizes_param = [(0.3, 0.15), (0.2, 0.09), (0.1, 0.05), (0.075, 0.03)]
palm_width_param = 0.15
finger_size_param = (0.085, 0.015)


# This function calls each of the set_transform_xxx functions, and puts the results
# in a list (the gripper - the last element - is a list)
arm_longer = create_arm_geometry(base_size_param, link_sizes_param, palm_width_param, finger_size_param)

# Set the angles of the arm
angles_start = [-np.pi/4.0, -np.pi/4, 1.2 * np.pi/4, -1 * np.pi/8, [-np.pi/3.0, np.pi/6.0, -np.pi/6.0]]
set_angles_of_arm_geometry(arm_longer, angles_start)

target_longer = np.array([0.3, -0.25])

In [ ]:
# Plot arm with target
fig, axs = plt.subplots(1, 1, figsize=(8, 8))
plot_complete_arm(axs, arm_longer, get_matrices_all_links(arm_longer))
axs.plot(target_longer[0], target_longer[1], '+g', markersize=20)

In [ ]:
# Do the optimization
angles_optimized = do_fmin(angles_start, arm_longer, target_longer)
set_angles_of_arm_geometry(arm_longer, angles_optimized)

In [ ]:
# Plot arm with target
fig, axs = plt.subplots(1, 1, figsize=(8, 8))
plot_complete_arm(axs, arm_longer, get_matrices_all_links(arm_longer))
axs.plot(target_longer[0], target_longer[1], '+g', markersize=20)

In [ ]:
assert(np.isclose(distance_to_goal(arm_geometry, target), 0.0, atol=0.01))

In [ ]:
grader.check("generalization")

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [ ]:

# List of names (creates a set)
worked_with_names = {"not filled out"}
# List of URLS (creates a set)
websites = {"not filled out"}
# Approximate number of hours, including lab/in-class time
hours = -1.5

# for all row, column in all_indices_from_where
#.   if this is the column for wrist torque 
#.      print(f"Row: {r}, Time step: {c // n_time_steps} Successful y/n: {pick_data[r, -1] == 1}, value: {pick_data[r, c]}")

In [ ]:
grader.check("hours_collaborators")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Submit through gradescope, HWK 3 Matrices and Optimization

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)